In [1]:
import numpy as np

def lfsr(initial_state, feedback_taps, num_cycles):
    state = initial_state
    output = []
    for _ in range(num_cycles):
        output.append(state)
        new_bit = sum([state[tap] for tap in feedback_taps]) % 2
        state = np.roll(state, 1)
        state[0] = new_bit
    return output

def misr(scan_chain_states, feedback_taps):
    misr_state = np.array([0, 0, 0, 0, 0])
    misr_outputs = []
    for state in scan_chain_states:
        misr_state = np.roll(misr_state, 1)
        misr_state[0] = state[0]
        for tap in feedback_taps:
            misr_state[0] ^= misr_state[tap]
        misr_outputs.append(np.copy(misr_state))
    return misr_outputs

# Define parameters
num_datasets = 100
num_cycles = 10
misr_feedback_taps = [4, 2]  # Example feedback taps for the MISR

# Generate datasets
for i in range(num_datasets):
    initial_state = np.random.randint(2, size=10)  # Random 10-bit initial state
    feedback_taps = np.random.randint(1, 10, 2).tolist()  # Random feedback taps for the LFSR

    # Generate scan chain states using LFSR
    scan_chain_states = lfsr(initial_state, feedback_taps, num_cycles)

    # Generate MISR outputs for the scan chain states
    misr_outputs = misr(scan_chain_states, misr_feedback_taps)

    print(f"Dataset {i+1}:")
    print("Scan Chain States:")
    for state in scan_chain_states:
        print("".join(map(str, state.astype(int))))
    print("MISR Outputs:")
    for output in misr_outputs:
        print("".join(map(str, output.astype(int))))
    print("-" * 30)


Dataset 1:
Scan Chain States:
1101111101
1110111110
1111011111
0111101111
0011110111
0001111011
1000111101
1100011110
0110001111
0011000111
MISR Outputs:
10000
11000
01100
10110
11011
01101
00110
00011
10001
01000
------------------------------
Dataset 2:
Scan Chain States:
0101111000
1010111100
0101011110
1010101111
0101010111
0010101011
0001010101
0000101010
1000010101
0100001010
MISR Outputs:
00000
10000
01000
00100
00010
10001
01000
10100
11010
01101
------------------------------
Dataset 3:
Scan Chain States:
1111011010
1111101101
0111110110
1011111011
0101111101
0010111110
0001011111
0000101111
1000010111
1100001011
MISR Outputs:
10000
11000
11100
01110
00111
10011
11001
11100
01110
10111
------------------------------
Dataset 4:
Scan Chain States:
1011011000
0101101100
1010110110
1101011011
0110101101
1011010110
1101101011
1110110101
1111011010
0111101101
MISR Outputs:
10000
01000
00100
10010
11001
01100
00110
00011
00001
00000
------------------------------
Dataset 5:
Scan Chai

In [7]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

# Define parameters
input_dim = 10  # 5-bit MISR output + 5-bit cycle number
output_dim = 10  # 10-bit scan chain
d_model = 64  # Dimensionality of the model
num_heads = 4  # Number of attention heads
ff_dim = 128  # Dimensionality of the feed-forward network

# Multi-head self-attention mechanism
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // self.num_heads
        self.query_dense = layers.Dense(units=d_model)
        self.key_dense = layers.Dense(units=d_model)
        self.value_dense = layers.Dense(units=d_model)
        self.dense = layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(tf.cast(self.depth, tf.float32))
        attention_weights = tf.nn.softmax(scaled_attention, axis=-1)
        context = tf.matmul(attention_weights, value)
        context = tf.transpose(context, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(context, (batch_size, -1, self.d_model))

        outputs = self.dense(concat_attention)


        # Inside the TransformerBlock's call method
def call(self, inputs, training=None):
    x = layers.Dense(self.d_model)(inputs)  # Add this line to transform the input shape
    attn_output = self.att(x)  # Use the transformed input x
    out1 = self.layernorm1(x + attn_output)
    ...

        return outputs

# Transformer block
class TransformerBlock(layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        attn_output = self.att(inputs)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)


# Add an embedding layer
embedding_layer = layers.Dense(d_model)

# Build the model
inputs = layers.Input(shape=(None, input_dim))  # Adjusted the input shape to be more flexible with sequence length.
x = TransformerBlock(d_model, num_heads, ff_dim)(inputs)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(output_dim, activation='sigmoid')(x)



model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam')


ValueError: ignored

In [5]:
# ...[Your code for the lfsr, misr functions, and data generation]...

# Prepare the data for the model
input_data = []
output_data = []

for i in range(num_datasets):
    for cycle in range(num_cycles):
        cycle_number = [int(x) for x in format(cycle, '05b')]  # Convert to 5-bit representation
        input_data.append(cycle_number + misr_outputs[cycle].tolist())
        output_data.append(scan_chain_states[cycle].tolist())

input_data = np.array(input_data)
output_data = np.array(output_data)

# Reshape for the model
input_data = input_data.reshape((-1, 1, 10))
output_data = output_data.reshape((-1, 10))

# Define the model as previously mentioned
# ...[Your model definition code]...

# Train the model
model.fit(input_data, output_data, epochs=10, batch_size=32)

# You can then evaluate the model's performance or save the model for further use.


Epoch 1/10


ValueError: ignored